This demos how to use sqlalchemy to input data into postgres.

In [7]:
import os
from os.path import expanduser
import pandas as pd
import pandas.io.sql as pd_sql
from functions.data import grades_converter
from functions.query import connection_fetch_close

project_name = os.getcwd().split('/')[-1]
with open(expanduser('.db/postgres.json')) as f:
    cred = json.load(f)
    username = cred.get("user")
    password = cred.get("password")
    host = cred.get("host")
    port = cred.get("port")

db_name = "mountain_project"
connection_str = f'postgres://{username}:{password}@{host}:{port}/{db_name}' # sqlalchemy connection args

# Move Data to Postgres

In [2]:
grades_converter_df = pd.DataFrame(list(grades_converter.items()), columns=['rating', 'grade'])
grades_converter_df.to_sql('grades_converter', connection_str, index=False)

# Check

In [3]:
query = """
SELECT *
FROM grades_converter
;
"""
rating_to_grade_df = pd_sql.read_sql(query, connection_str)
rating_to_grade_df

,rating,grade
0,V-easy,-1.00
1,V-easy PG13,-1.00
2,5.9 V-easy,-1.00
3,V-easy R,-0.75
4,V0-,-0.25
...,...,...
115,V13,13.00
116,V13 PG13,13.00
117,V13 R,13.25
118,V13-14 PG13,13.50


In [4]:
# Check data types

query = """
SELECT data_type
FROM information_schema.columns
WHERE table_name = 'grades_converter';
"""

results = connection_fetch_close(query, cred, dbname='mountain_project')
results

[('text',), ('double precision',)]